# Python assembler for the microprocessor made in EE2016 (Jul-Nov 2017)
## Rajat Vadiraj Dwaraknath EE16B033

The cells which are by default raw cells are used to test the modules defined above them. Comment out the writefile magic at the top of each cell before using these test cells.

The basic instruction format is:

4 bit reg address of operand2; 4 bit reg address of operand1; 4 bit reg address of destination; 4 bit opcode;

operand2 is optional.

Mnemonics which end with the character 'I' are immediate instructions. They usually substitute one of the register values with a literal value, which can take upto 8 bits, after 4 for opcode and 4 for operand1.

Labels are referenced with the $ symbol.

Literals are created with the # symbol.

This assembler currently does not enforce syntax.

In [1]:
%%writefile pythonAssemble.py
# Python assembler for microprocessor made in EE2016 (Jul-Nov 2017)
# Author: Rajat Vadiraj Dwaraknath EE16B033
import re


Overwriting pythonAssemble.py


In [2]:
%%writefile pythonAssemble.py -a

DATA_LOC = 128
MEMORY_VAR = "memory"
MACRO_NAME = "writeRam"

OPCODE = {
    "ADD":0,
    "SUB":1,
    "AND":2,
    "OR":3,
    "LS":4,
    "RS":5,
    "LDI":6,
    "STI":7,
    "MOV":8,
    "LD":9,
    "ST":10,
    "CMP":11,
    "BCI":12,
    "BNEI":13,
    "BI":14,
}

Appending to pythonAssemble.py


In [3]:
%%writefile pythonAssemble.py -a


def removeEmptyLines(lines):
    return [line for line in lines if line.strip()!=""]

Appending to pythonAssemble.py


In [4]:
%%writefile pythonAssemble.py -a


def getLabels(lines):
    labs = {"PROGRAM":1, "DATA":DATA_LOC}
    reserved = ["PROGRAM", "DATA"]
    for i,line in enumerate(lines):
        if ":" in line:
            l = line.split(":")[0].strip()
            if l not in reserved:
                labs[l] = i;
    return labs

Appending to pythonAssemble.py


In [5]:
%%writefile pythonAssemble.py -a


def compileLine(line,labels):
    s = ""
    if ":" in line:
        line = line.split(":")[1].strip()
        
    ops = filter(None, re.split("[, ]+", line))
    if len(ops)==0:
        return ""
    opcode = ops[0]
    s = "{:04b}".format(OPCODE[ops[0]])
    if opcode[-1] == 'I':
        for op in ops[1:]:
            if op[0]=='R':
                s = "{:04b}".format(int(op[1:])) + s
            elif op[0]=='#':
                s = "{:b}".format(int(op[1:])) + s
            elif op[0]=='$':
                s = "{:b}".format(labels[op[1:]]) + s
    else:        
        for op in ops[1:]:
            s = "{:04b}".format(int(op[1:])) + s
            
    s = "0"*(16-len(s))+s
    s = "16'b"+s
    return s
    

Appending to pythonAssemble.py


In [6]:
%%writefile pythonAssemble.py -a


def compileIntoM(text):
    lines = text.split("\n")
    lines = [l.strip().upper() for l in lines]
    lines = removeEmptyLines(lines)
    
    labels = getLabels(lines)
    
    
    i=1
    line = lines[i]
    output = "`define {} \\\n".format(MACRO_NAME)
    while line != 'DATA:':
        
        instcode = compileLine(line,labels)
        output += "{0}[{1}] = {2};".format(MEMORY_VAR,i,instcode) + " \\\n"
        
        i+=1
        line = lines[i]
        
    for j,line in enumerate(lines[i+1:]):
        output += "{0}[{1}] = {2};".format(MEMORY_VAR,j+DATA_LOC,line.strip()) + " \\\n"
        
    return output

Appending to pythonAssemble.py


In [7]:
%%writefile pythonAssemble.py -a


import sys
filename = sys.argv[1]
outname = "code.v"
with open(filename) as f:
    text = f.read()
    compiledstring = compileIntoM(text)

with open(outname,"w") as f:
    f.write(compiledstring)

Appending to pythonAssemble.py


In [8]:
%%writefile test.myasm
PROgRAM:
LDI R0,$DATA
LDI R13,#1
LD R1,R0
Addstmt : ADD R0  ,R0,R13
LD R2,R0

loop: ADD R5,R5,R1
SUB R2,R2,R13
BNEI $loop

DATA:
15
4

Overwriting test.myasm


In [9]:
!python pythonAssemble.py test.myasm